# Kreiranje neuronske mreže

In [32]:
import torch
from torch import nn
import torch.nn.functional as F

In [33]:
class Covertype(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Linear(54,100)
        self.act1=nn.ReLU()
        self.layer2=nn.Linear(100,50)
        self.act2=nn.ReLU()
        self.layer3=nn.Linear(50,7)
        self.act3=nn.Softmax()
        
    def forward(self,x):
        x=self.act1(self.layer1(x))
        x=self.act2(self.layer2(x))
        x=F.softmax(self.layer3(x), dim=1)
        return x

In [34]:
from sklearn.datasets import fetch_covtype
coverType = fetch_covtype()
#podjela podataka na ulazne karakteristike i ciljne vrijednosti
X=coverType.data
Y=coverType.target

from sklearn.model_selection import train_test_split
#dijelimo podatke u trening i test skupove
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [35]:
X_train_t=torch.tensor(X_train).float()
X_test_t=torch.tensor(X_test).float()

#niz brojeva se prebacuje u tenzor nx1
Y_train_t=torch.tensor(Y_train).float().reshape(-1,1)
Y_test_t=torch.tensor(Y_test).reshape(-1,1)

In [36]:
from torch.utils.data import TensorDataset, DataLoader

#TensorDataset stvara ulazni podatak spajajući 2 tenzora
train_dataset=TensorDataset(X_train_t, Y_train_t)
test_dataset=TensorDataset(X_test_t, Y_test_t)

#dataLoader nam omogucava da iteriramo kroz podatke u grupama(batch)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32)

In [37]:
net=Covertype() #instanca neuronske mreze
loss_fn=nn.NLLLoss()#definisemo funkciju greske

In [38]:
from sklearn.metrics import precision_score, recall_score

#testiramo model
net.eval()#mreza je u rezimu za testiranje
def test_model(net, loss_fn, test_loader):
    predictions=[] #lista vektora
    true_labels=[]
    total_loss=0
    for x, y in test_loader:
        #ulazne podatke pretvaramo u float tip a izlazne klase u long integer
        x,y=x.float(), y.long().squeeze()
        
        #propagacija unaprijed, metoda forward klase covertype
        #daje nam predvidjen izlaz za dati ulazni podatak
        pred=net(x)
        
        #racunamo gresku
        loss=loss_fn(pred,y-1)#klase su indeksirane od 0 pa je y-1
        total_loss+=loss.item()#ukupna greska
        
        #trazimo vrijednost one vjerovatnove koja je najveca
        #predicted cuva koju klasu ona oznacava
        max, predicted=torch.max(pred.data, 1)
        #dodajemo oznaku klase u vektor(tenzor->vektor)
        predictions.extend(predicted.numpy())
        true_labels.extend((y-1).numpy())
        
    precision=precision_score(true_labels,predictions, average='weighted', zero_division=1)
    recall = recall_score(true_labels, predictions, average='weighted', zero_division=1)
    return precision, recall

precision,recall=test_model(net,loss_fn,test_loader)
print("Preciznost i odziv prije treninga, respektivno:")
print(precision)
print(recall)    

Preciznost i odziv prije treninga, respektivno:
0.9664816398058695
0.030042253642332813


Trening mreze:

In [39]:
#Adam optimizacija, learning rate je 0.001
optimizer = torch.optim.Adam(net.parameters(),1e-3,weight_decay=1e-5) 

EPOCHS=2#proizvoljan broj epoha
net.train()#mreza je u rezimu treniranja

for i in range(EPOCHS):
    for x, y in train_loader:
        x,y=x.float(), y.long().squeeze()
        #propagacija unaprijed da bismo dobili izlaz
        preds=net(x)
        #racunanje greske
        loss=loss_fn(preds, y-1)
        #postavljanje prethodnih gradijenata na nulu
        optimizer.zero_grad()
        #racunanje gradijenta gubitka
        loss.backward()
        #azuriranje parametara neuronske mreze
        optimizer.step()

Testiranje poslije treninga

In [40]:
precision, recall=test_model(net,loss_fn,test_loader)

print("Preciznost: ", precision)
print("Odziv: ", recall)

Preciznost:  0.7501899412122015
Odziv:  0.48621808387046805


# Definisanje validacionog skupa i early stopping

-validacioni skup mora da sadrzi podatke kojih nema ni u trening ni u testnom skupu jer zelimo da vidimo kako model radi sa ulazima koje nije vidio ranije

-algoritam se izvrsava sve dok performanse ne pocnu da se pogorsavaju na tom skupu

In [41]:
coverType=fetch_covtype()

X=coverType.data
Y=coverType.target

#30% odvajamo za testni skup jer ćemo ga dijeliti na testni i validacioni
X_train, X_temporary, Y_train, Y_temporary=train_test_split(X,Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test=train_test_split(X_temporary, Y_temporary, test_size=0.5, random_state=42)

X_val_t=torch.tensor(X_val).float()
Y_val_t=torch.tensor(Y_val).reshape(-1,1)


In [42]:
#dataset u vidu tenzora
val_dataset=TensorDataset(X_val_t, Y_val_t)
val_loader=DataLoader(val_dataset, batch_size=32,shuffle=True)


In [53]:
def Early_stopping(net, train_loader, val_loader, loss_fn, optimizer):
    EPOCHS=10
    net.train()
    best_loss=None
    allowEpochs=2
    badEpochs=0
    
    for i in range(EPOCHS):
        print("Izvrsava se {}. epoha...".format(i+1))
        #treniramo mrezu
        for x,y in train_loader:
            x,y=x.float(), y.long().squeeze()
            predictions=net(x)
            loss=loss_fn(predictions,y-1)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        #racunanje ukupne greske
        value_loss=0
        for x,y in val_loader:
            x,y=x.float(), y.long().squeeze()
            preds=net(x)
            loss=loss_fn(preds,y-1)
            value_loss+=loss.item()
            
        #ovime dobijamo prosjecni gubitak po primjeru
        value_loss/=len(val_loader)
        
        #provjera da li je taj gubitak bolji od prethodnog
        if not best_loss:
            best_loss=value_loss
        elif value_loss<best_loss:
            best_loss=value_loss
            badEpochs=0
        else:
            badEpochs+=1
            if badEpochs>=allowEpochs:
                print("Nakon {}-e i {}-e epohe dolazi do pogorsanja rezultata te se prekida trening.".format(i+1, i))
                print("Broj losih epoha:", badEpochs)
                break

In [54]:
Early_stopping(net,train_loader,val_loader,loss_fn,optimizer)

Izvrsava se 1. epoha...
Izvrsava se 2. epoha...
Izvrsava se 3. epoha...
Nakon 3-e i 2-e epohe dolazi do pogorsanja rezultata te se prekida trening.
Broj losih epoha: 2


# Pretraga hiperparametara

In [55]:
coverType=fetch_covtype()
X=coverType.data
Y=coverType.target

X_train, X_temporary, Y_train, Y_temporary=train_test_split(X,Y,test_size=0.3, random_state=42)
X_val,X_test,Y_val,Y_test=train_test_split(X_temporary, Y_temporary, test_size=0.5, random_state=42)

In [56]:
X_train_t=torch.tensor(X_train).float()
Y_train_t=torch.tensor(Y_train).float().reshape(-1,1)

X_val_t=torch.tensor(X_val).float()
Y_val_t=torch.tensor(Y_val).reshape(-1,1)


In [57]:
class covType(nn.Module):
    def __init__(self,hidden):
        super().__init__()
        self.layer1=nn.Linear(54,hidden)
        self.act1=nn.ReLU()
        self.layer2=nn.Linear(hidden, hidden)
        self.act2=nn.ReLU()
        self.layer3=nn.Linear(hidden,7)
        self.act3=nn.Softmax()
        
    def forward(self,x):
        x=self.act1(self.layer1(x))
        x=self.act2(self.layer2(x))
        x=F.softmax(self.layer3(x), dim=1)
        return x


In [58]:
#hiperparametri:(pored navedenih, moze se varirati sa optimizerima, aktivacionim funkcijama)
learning_rate=[0.001, 0.01]
hidden_layers=[60,100]
batch_sizes=[64,128]

best_precision=0.0
best_recall=0.0
best_model=None
best_hyperparemeters=None

for i in learning_rate:
    for j in hidden_layers:
        for k in batch_sizes:
            train_dataset=TensorDataset(X_train_t, Y_train_t)
            val_dataset=TensorDataset(X_val_t, Y_val_t)
            train_loader=DataLoader(train_dataset, batch_size=k, shuffle=True)
            val_loader=DataLoader(val_dataset, batch_size=k, shuffle=True)
            
            net=covType(j)
            loss_fn=nn.NLLLoss()
            optimizer=torch.optim.Adam(net.parameters(), lr=i)
            
            Early_stopping(net, train_loader, val_loader, loss_fn, optimizer)
            
            precision, recall=test_model(net, loss_fn, val_loader)
            if precision>best_precision:
                best_precision=precision
                best_recall=recall
                best_model=net
                best_parameters=[i, j, k]

Izvrsava se 1. epoha...
Izvrsava se 2. epoha...
Izvrsava se 3. epoha...
Nakon 3-e i 2-e epohe dolazi do pogorsanja rezultata te se prekida trening.
Broj losih epoha: 2
Izvrsava se 1. epoha...
Izvrsava se 2. epoha...
Izvrsava se 3. epoha...
Nakon 3-e i 2-e epohe dolazi do pogorsanja rezultata te se prekida trening.
Broj losih epoha: 2
Izvrsava se 1. epoha...
Izvrsava se 2. epoha...
Izvrsava se 3. epoha...
Izvrsava se 4. epoha...
Nakon 4-e i 3-e epohe dolazi do pogorsanja rezultata te se prekida trening.
Broj losih epoha: 2
Izvrsava se 1. epoha...
Izvrsava se 2. epoha...
Izvrsava se 3. epoha...
Izvrsava se 4. epoha...
Izvrsava se 5. epoha...
Izvrsava se 6. epoha...
Izvrsava se 7. epoha...
Nakon 7-e i 6-e epohe dolazi do pogorsanja rezultata te se prekida trening.
Broj losih epoha: 2
Izvrsava se 1. epoha...
Izvrsava se 2. epoha...
Izvrsava se 3. epoha...
Izvrsava se 4. epoha...
Nakon 4-e i 3-e epohe dolazi do pogorsanja rezultata te se prekida trening.
Broj losih epoha: 2
Izvrsava se 1. e

In [61]:
print("Najbolje perfomanse mreze dobijamo pri parametrima:\nlearning rate={}, hidden layers={}, batch_size={}:\n".format(i, j, k))
print("Preciznost je", best_precision)
print("Odziv je", best_recall)

Najbolje perfomanse mreze dobijamo pri parametrima:
learning rate=0.01, hidden layers=100, batch_size=128:

Preciznost je 0.9954200140420941
Odziv je 0.0046011565999632825
